# Mass–Spring Physics and Data Generation

Welcome! In this notebook we gently explore the physics behind a mass attached to a spring and create the synthetic data that will feed our neural networks. Feel free to read each Markdown cell slowly and run the code blocks in order. Nothing here requires prior experience with physics-informed models.

We will:

1. Describe the physical system in everyday language.
2. Solve the differential equation that governs the motion using a reliable numerical solver.
3. Generate both clean and noisy datasets so we can later test how different neural networks behave.
4. Visualize the displacement and (optionally) the velocity of the mass over time.


## 1. Import libraries and set default parameters

We keep the imports minimal: NumPy for numerical operations, SciPy for solving the differential equation, and Matplotlib for plotting. The parameters `m`, `c`, and `k` describe the mass, damping, and spring stiffness respectively. Feel free to experiment with these values later!

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pathlib import Path

# Make plots a little prettier and larger
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 5)


def find_project_root(start: Path) -> Path:
    '''Locate the project directory that contains the notebooks and src folder.'''
    for candidate in [start, *start.parents]:
        if (candidate / 'figures').exists() and (candidate / 'src').exists():
            return candidate
    raise FileNotFoundError('Could not locate the project root. Please run the notebook from inside the repository.')

ROOT = find_project_root(Path.cwd())
FIGURE_DIR = ROOT / 'figures'
FIGURE_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR = ROOT / 'notebooks'
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Physical parameters for the mass-spring system
m = 1.0  # mass in kilograms
k = 1.0  # spring stiffness in newtons per meter
c = 0.1  # damping coefficient (0 = no damping)

print(f"Using parameters: m={m}, c={c}, k={k}")

## 2. Describe the differential equation

The displacement of the mass, noted as $x(t)$, follows the second-order differential equation

```
m * x''(t) + c * x'(t) + k * x(t) = 0
```

- $x'(t)$ is the velocity (how fast the displacement is changing).
- $x''(t)$ is the acceleration (how fast the velocity is changing).

Because the equation involves second derivatives, we rewrite it as a system of first-order equations so that `solve_ivp` can handle it. We model a simple initial condition: the mass starts with a displacement of 1 meter and zero initial velocity. This gives us a nice oscillation to look at.

In [ ]:
def mass_spring_system(t, y, m, c, k):
    '''Return the derivatives for the mass-spring system.'''
    x, v = y  # displacement and velocity
    dxdt = v
    dvdt = -(c / m) * v - (k / m) * x
    return [dxdt, dvdt]

# Initial conditions: start displaced by 1 meter with zero velocity
initial_state = [1.0, 0.0]

# Time span and evaluation grid for the solver
t_start, t_end = 0.0, 10.0
num_points = 1000
time_eval = np.linspace(t_start, t_end, num_points)

solution = solve_ivp(
    fun=mass_spring_system,
    t_span=(t_start, t_end),
    y0=initial_state,
    t_eval=time_eval,
    args=(m, c, k),
    dense_output=True,
)

if not solution.success:
    raise RuntimeError("ODE solver failed: " + solution.message)

x_true = solution.y[0]
velocity_true = solution.y[1]

print(f"Solved the system on {num_points} time points.")

## 3. Create clean and noisy datasets

With the reference solution in hand we can prepare the datasets. The clean dataset contains the exact displacement values from the solver. To mimic noisy measurements we add Gaussian noise with a small standard deviation. This helps us later show how neural networks cope with imperfect data.

In [ ]:
noise_level = 0.02  # 2% of the signal amplitude
np.random.seed(42)  # for reproducibility

signal_amplitude = np.max(np.abs(x_true))
noise_std = noise_level * signal_amplitude
x_noisy = x_true + np.random.normal(scale=noise_std, size=x_true.shape)

print(f"Noise standard deviation: {noise_std:.4f}")

# Bundle arrays into a dictionary for later use. Using plain NumPy keeps
# dependencies minimal while still being easy to save to disk later.
data = {
    'time': time_eval,
    'displacement_clean': x_true,
    'displacement_noisy': x_noisy,
    'velocity_clean': velocity_true,
}

# Show the first five entries to double-check the values
for i in range(5):
    print(f"t={data['time'][i]:.3f} s | x_clean={data['displacement_clean'][i]:.4f} | x_noisy={data['displacement_noisy'][i]:.4f}")


## 4. Plot the displacement and velocity

The plots below show the clean displacement curve and the noisy measurements. The velocity plot is optional but helps visualize how the mass slows down over time due to damping.

In [ ]:
fig, ax = plt.subplots()
ax.plot(time_eval, x_true, label='Clean displacement', linewidth=2)
ax.scatter(time_eval, x_noisy, label='Noisy measurements', s=10, alpha=0.4)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Displacement [m]')
ax.set_title('Mass–spring displacement over time')
ax.legend()
fig.tight_layout()
fig.savefig(FIGURE_DIR / 'displacement.svg', dpi=150, format='svg')
fig.savefig(FIGURE_DIR / 'noisy_samples.svg', dpi=150, format='svg')
plt.show()

fig, ax = plt.subplots()
ax.plot(time_eval, velocity_true, color='tab:green', linewidth=2)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Velocity [m/s]')
ax.set_title('Velocity of the mass over time')
fig.tight_layout()
fig.savefig(FIGURE_DIR / 'velocity.svg', dpi=150, format='svg')
plt.show()



## 5. Package the data for reuse

We wrap the logic above into a small helper function. You can import this function or copy it into other notebooks so that you can regenerate the clean and noisy curves whenever you need them.


In [ ]:

def generate_mass_spring_data(
    m: float = 1.0,
    c: float = 0.1,
    k: float = 1.0,
    noise_level: float = 0.02,
    t_start: float = 0.0,
    t_end: float = 10.0,
    num_points: int = 1000,
    seed: int | None = 42,
):
    """Return clean and noisy displacement data for the mass-spring system."""
    time_eval = np.linspace(t_start, t_end, num_points)

    # Solve the differential equation using SciPy.
    solution = solve_ivp(
        mass_spring_system,
        t_span=(t_start, t_end),
        y0=[1.0, 0.0],
        t_eval=time_eval,
        args=(m, c, k),
    )
    x_true = solution.y[0]
    velocity_true = solution.y[1]

    # Create reproducible noise to mimic imperfect measurements.
    rng = np.random.default_rng(seed)
    signal_amplitude = np.max(np.abs(x_true))
    noise_std = noise_level * signal_amplitude
    x_noisy = x_true + rng.normal(loc=0.0, scale=noise_std, size=x_true.shape)

    return {
        "time": time_eval,
        "displacement_clean": x_true,
        "displacement_noisy": x_noisy,
        "velocity_clean": velocity_true,
        "params": {"m": m, "c": c, "k": k},
        "noise_level": noise_level,
    }


# Generate a dictionary right away so the data is easy to inspect.
dataset = generate_mass_spring_data()
print(f"Clean displacement shape: {dataset['displacement_clean'].shape}")
print(f"Noisy displacement shape: {dataset['displacement_noisy'].shape}")


## 6. Recap

- We solved the damped mass–spring differential equation numerically.
- We generated 1,000 time samples and recorded both the clean displacement and a noisy version.
- We exported simple SVG plots to the `figures/` folder so the GitHub Pages site can reuse them.

You are now ready to open the second notebook where we train neural networks on this data. Keep this notebook handy if you want to experiment with different parameter values or noise levels.